# Dask Array

In [1]:
import dask.array as da
import h5py
import numpy as np
import dask
import pandas as pd

In [2]:
!pip install graphviz 

In [3]:
dask.__version__

'2021.04.0'

In [ ]:
!pip install --user --upgrade dask

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [4]:
with h5py.File("demo.h5", "w") as hdf:
    hdf.create_dataset('arr', data=np.random.normal(0, 1, size = (1000, 300_000)))

In [5]:
hdf = h5py.File("demo.h5", "r")
dset = hdf["arr"]
arr = da.from_array(dset, chunks=(1000, 30000))
arr

dask.array<array, shape=(1000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

In [6]:
%%time
arr.mean()

Wall time: 2 ms


dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [7]:
%%time
arr.mean().compute()

Wall time: 746 ms


-1.8351240198487705e-05

In [8]:
arr1 = arr * 2
s = arr1.sum()
m = arr1.mean()

In [9]:
%%time
arr1 = arr * 2
s = arr1.sum()
print(s.compute())
m = arr1.mean()
print(m.compute())

-11010.744119092622
-3.670248039697541e-05
Wall time: 2 s


In [10]:
%%time
arr1 = arr * 2
s = arr1.sum()
m = arr1.mean()
dask.compute(
    s, m
)

Wall time: 1.01 s


(-11010.744119092622, -3.670248039697541e-05)

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [12]:
%%time
arr_np = np.random.normal(0, 1, size=(1000, 300000))
(arr_np ** 2).sum()

Wall time: 7.34 s


299957162.3801849

In [13]:
%%time
arr_da = da.random.normal(0, 1, size=(1000, 300000))
(arr_da ** 2).sum().compute()

Wall time: 1.6 s


300013956.20837116

In [14]:
np.mean(arr_da).compute()

-7.965942109263818e-05

In [15]:
da.mean(arr_da)

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

## Лабораторная работа 11

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений (расчет средних значений, расчет косинусной близости и т.д.) проводятся над `dask.array` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.array` к `numpy.array` или `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.array` (то есть то, что было получено до вызова `compute`, а не после).

In [1]:
import h5py
import dask
import numpy as np
import pandas as pd
import numpy.ma as ma
import dask.array as da
from scipy.spatial import distance
from sklearn.metrics import accuracy_score

1\. Считайте датасет `embeddings` из файла `recipe_embeddings.h5` в виде `dask.array`. Выведите на экран основную информацию о массиве: размер, форму, тип, количество и размер сегментов. 

In [2]:
hdf = h5py.File("recipe_embeddings.h5", "r")
dset = hdf["embeddings"]
arr_embeddings = da.from_array(dset)
arr_embeddings

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

2\. Посчитайте и выведите на экран среднее значение всех элементов массива. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

Пусть $M$ - количество строк в массиве, $N$ - количество столбцов в массиве, `chunks=(r,c)`. Сравните несколько вариантов:
* $r=M$, $с \ll N$ , 
* $r \ll M$, $c=N$ 
* $r = M$, $c = N$ 
* значения $r, c$ по умолчанию.

Выберите наиболее оптимальные значения $r$ и  $c$ в смысле скорости вычислений и далее продолжайте работу с ними.

In [3]:
arr_embeddings_1 = da.from_array(dset, chunks=(1200000, 156))
arr_embeddings_1

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(1200000, 156), chunktype=numpy.ndarray>

In [4]:
%%time
arr_embeddings_1.mean().compute()

Wall time: 1.84 s


0.0023777517

In [5]:
arr_embeddings_2 = da.from_array(dset, chunks=(120000, 312))
arr_embeddings_2

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(120000, 312), chunktype=numpy.ndarray>

In [6]:
%%time
arr_embeddings_2.mean().compute()

Wall time: 556 ms


0.0023777566

In [7]:
arr_embeddings_3 = da.from_array(dset, chunks=(1200000, 312))
arr_embeddings_3

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(1200000, 312), chunktype=numpy.ndarray>

In [8]:
%%time
arr_embeddings_3.mean().compute()

Wall time: 729 ms


0.0023777678

3\. Опишите пространство, в котором расположены эмбеддинги, посчитав минимальное и максимальное значение для каждой из координат. Сведите результаты в таблицу `pd.DataFrame`, состоящую из двух строк и 312 столбцов. Задайте индексы строк "min" и "max". Названия столбцов сделайте вида $x_i$. Выведите полученную таблицу на экран.

Решите задачу двумя способами. В первом варианте сделайте два вызова метода `compute` для расчета каждого из векторов максимальных и минимальных значений. Во втором варианте сделайте один вызов функции `dask.compute` для одновременного расчета двух векторов. Сравните время выполнения двух решений.

In [9]:
%%time

mx = np.max(arr_embeddings_2, axis=0).compute()
mn = np.min(arr_embeddings_2, axis=0).compute()

df_emb_1 = pd.DataFrame(np.array([mx, mn]),
                   columns=np.linspace(1,312,312).astype('int'), index=['max', 'min'])
df_emb_1

Wall time: 927 ms


,1,2,3,4,5,6,7,8,9,10,...,303,304,305,306,307,308,309,310,311,312
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348


In [10]:
%%time

mx_2 = np.max(arr_embeddings_2, axis=0)
mn_2 = np.min(arr_embeddings_2, axis=0)

comp = dask.compute(mx_2, mn_2)

df_emb_2 = pd.DataFrame(np.array([comp[0], comp[1]]),
                   columns=np.linspace(1,312,312).astype('int'), index=['max', 'min'])
df_emb_2

Wall time: 499 ms


,1,2,3,4,5,6,7,8,9,10,...,303,304,305,306,307,308,309,310,311,312
max,0.135038,0.076125,0.157854,0.030987,0.101192,0.111774,0.147497,0.173821,0.099808,0.115573,...,0.119518,0.197589,0.113135,0.136490,0.162921,0.099021,0.086653,0.158176,0.166968,0.048967
min,-0.132803,-0.149056,-0.094468,-0.191697,-0.114229,-0.114341,-0.096039,-0.115178,-0.157275,-0.116715,...,-0.103254,-0.122285,-0.149789,-0.127703,-0.094802,-0.119690,-0.141425,-0.123732,-0.081543,-0.227348


4\. Найдите вектор $x \ne x_{256}$ из набора данных, ближайший к вектору $x_{256}$ в смысле метрики $L_1$. Выведите на экран первые 10 координат вектора $x$.

$$d_1(\textbf{x},\textbf{y})=\sum_{k=1}^{n}{|x_k - y_k|}, \textbf{x}, \textbf{y} \in \mathbb{R}^n$$

In [11]:
x_256 = arr_embeddings[255,:]
arr_sum = da.sum(abs(arr_embeddings - x_256), axis=1)
min_value = da.ma.masked_where(arr_sum == 0, arr_sum).min() # Проверка, на то, что значение метрики L1 !=0 и, соответсвенно, x != x_256

ind, = da.where(arr_sum == min_value)  # Находим индекс вектора из набора данных, который ближайший к вектору 𝑥_256  в смысле метрики L1
arr_embeddings[ind,:10].compute()

C:\Users\sanha\AppData\Roaming\Python\Python38\site-packages\dask\array\slicing.py:1084: PerformanceWarning: Increasing number of chunks by factor of 12
  p = blockwise(


array([[-0.01873741, -0.07140347,  0.02849776, -0.10885686,  0.03978413,
        -0.00868603,  0.03658793,  0.02858754, -0.07105186, -0.01334546]],
      dtype=float32)

5\. Рецепты разбиты на 4 группы. Загрузите маску для разбиения на группы из датасета `mask` из файла `recipe_embeddings.h5` в виде `dask.array`. Для каждой группы посчитайте и выведите на экран максимальное значение  нормы $\ell_1$ векторов рецептов, принадлежащих к этой группе. 

Подсказка: закодируйте маску принадлежности к группе при помощи метода кодирования one-hot encoding и воспользуйтесь механизмом распространения.

$$\ell_1: ||\textbf{x}||_1=\sum_{k=1}^{n}{|x_k|}, \textbf{x} \in \mathbb{R}^n$$

In [12]:
dset_m = hdf['mask']
mask_arr = da.from_array(dset_m)
mask_arr

dask.array<array, shape=(1200000,), dtype=int64, chunksize=(1200000,), chunktype=numpy.ndarray>

In [13]:
l1_metric = abs(arr_embeddings).sum(axis=1)
l1_metric

dask.array<sum-aggregate, shape=(1200000,), dtype=float32, chunksize=(107546,), chunktype=numpy.ndarray>

In [14]:
(l1_metric.reshape(-1,1) * da.eye(4)[mask_arr]).max(axis=0).compute()

array([13.31967735, 13.32409477, 13.31525993, 13.31915665])

6\. Пусть $X=[\textbf{x}_1,...\textbf{x}_M]^\top$ - матрица эмбеддингов рецептов размера $M\times N$, $W=[\textbf{w}_1,...,\textbf{w}_N]^\top$ - матрица коэффициентов некоторой модели машинного обучения размера $N\times 4$, $y=[y_1,...,y_M]^\top$ - вектор размера $M$, содержащий номера групп рецептов (метки классов). Тогда задачу классификации можно решить следующим образом: $$\hat{y_i} = argmax_j{<X_{i\cdot}, W_{\cdot j}>}$$ где $A_{i\cdot}$ обозначает $i$ строку матрицы, $A_{\cdot j}$ обозначает $j$ столбец матрицы, $\hat{y_i}$ - прогноз класса для рецепта $i$, $<\cdot, \cdot>$ - скалярное произведение векторов.

Инициализируйте матрицу $W$ случайным образом и получите прогнозы для всех рецептов при помощи этой матрицы и матрицы эмбеддингов. Подсчитайте и выведите на экран значение accuracy на основе полученных прогнозов $\hat{y}$ и правильных ответов $y$.

In [15]:
with h5py.File("coef_matrix.h5", "w") as hdf:
     hdf.create_dataset('arr', data=np.random.normal(0, 1, size = (312, 4)))

In [16]:
hdf = h5py.File("coef_matrix.h5", "r")
d_set = hdf['arr']

arr_coef = da.from_array(d_set)
arr_coef    # W

dask.array<array, shape=(312, 4), dtype=float64, chunksize=(312, 4), chunktype=numpy.ndarray>

In [17]:
arr_embeddings # X

dask.array<array, shape=(1200000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

In [18]:
y_hat = arr_embeddings.dot(arr_coef).argmax(axis=1) 
y_hat

dask.array<arg_agg-aggregate, shape=(1200000,), dtype=int64, chunksize=(107546,), chunktype=numpy.ndarray>

In [19]:
mask_arr # y_i

dask.array<array, shape=(1200000,), dtype=int64, chunksize=(1200000,), chunktype=numpy.ndarray>

In [20]:
accuracy = da.sum(y_hat == mask_arr) / y_hat.shape[0]
accuracy.compute()

0.09288666666666667

7\. Сингулярным разложением (SVD) матрицы $A$ размера $M\times N$ называется разложение вида $A = USV^\top$, где $U$ - матрица размера $M\times N$  ортонормированных векторов произведения $AA^\top$, $V^T$ - транспонированная матрица размера $N\times N$ ортонормированных векторов произведения $A^\top A$, $S$ - диагональная матрица сингулярных значений размера $N\times N$.

SVD может быть использовано для понижения размерности векторов. Для этого от матрицы $U$ оставляют первые $k$ столбцов $U_{\cdot,:k}$, от матрицы $S$ оставляют левый верхний квадрат размера $k\times k$ $S_{:k,:k}$ и вычисляется произведение $\hat{A} = U_{\cdot,:k}S_{:k,:k}$

Выберите эмбеддинги тех рецептов, которые относятся к группе с номеров 3, и уменьшите их размерность до 64 при помощи реализации алгоритма SVD из пакета `dask.array.linalg`. Выведите количество строк и столбцов полученного массива.

Примечание: после отбора рецепта, принадлежащих третьей группе, вызовите у полученного массива метод `compute_chunk_sizes`, чтобы `dask` обновил метаинформацию в этом массиве. 

In [21]:
group_3 = arr_embeddings[mask_arr==3].compute_chunk_sizes()
group_3

dask.array<getitem_variadic, shape=(10000, 312), dtype=float32, chunksize=(921, 312), chunktype=numpy.ndarray>

In [22]:
U, S, _ = da.linalg.svd(group_3)
k = 64
group_3_low_dim = U[:, :k].dot(da.diag(S[:k]))
group_3_low_dim.shape

(10000, 64)

8\. Используя эмбеддинги уменьшенной размерности, полученные в задании 6, посчитайте косинусное сходство между каждой парой рецептов третьей группы. Выведите матрицу косинусного сходства на экран.

In [23]:
group_3_low_dim
norm = da.linalg.norm(group_3_low_dim, axis=1)
norm

dask.array<pow, shape=(10000,), dtype=float64, chunksize=(921,), chunktype=numpy.ndarray>

In [24]:
emb_normed = group_3_low_dim / norm.reshape(-1,1)
emb_normed

dask.array<truediv, shape=(10000, 64), dtype=float64, chunksize=(921, 64), chunktype=numpy.ndarray>

In [25]:
cos_sim = emb_normed @ emb_normed.T
cos_sim.compute()

C:\Users\sanha\AppData\Roaming\Python\Python38\site-packages\dask\array\routines.py:446: PerformanceWarning: Increasing number of chunks by factor of 12
  out = blockwise(


array([[1.        , 0.77996178, 0.70962799, ..., 0.78566266, 0.72498332,
        0.70748998],
       [0.77996178, 1.        , 0.64223863, ..., 0.86709917, 0.61588697,
        0.65344302],
       [0.70962799, 0.64223863, 1.        , ..., 0.64595625, 0.87130853,
        0.87150268],
       ...,
       [0.78566266, 0.86709917, 0.64595625, ..., 1.        , 0.64296768,
        0.66372964],
       [0.72498332, 0.61588697, 0.87130853, ..., 0.64296768, 1.        ,
        0.91261633],
       [0.70748998, 0.65344302, 0.87150268, ..., 0.66372964, 0.91261633,
        1.        ]])

In [26]:
da.allclose(cos_sim, cos_sim.T).compute()

True

9\. Посчитайте и выведите на экран количество рецептов, для которых рецепт с индексом `242` входит число топ-5 ближайших рецептов в смысле косинусной близости. При поиске топ-5 рецептов для конкретного рецепта считайте, что он сам в это число не входит.

In [27]:
cos_sim.argtopk(5, axis=1).compute()

array([[   0, 8317,  715, 6056, 2307],
       [   1,  283, 6281, 3796, 6049],
       [   2, 4062, 1203,  855, 4131],
       ...,
       [9997, 6089,  960, 1950,  510],
       [9998, 1085, 7803, 4069,  452],
       [9999, 8878, 1361, 7997, 2944]], dtype=int64)

In [28]:
cos_sim.topk(5, axis=1).compute()

array([[1.        , 0.92920586, 0.91955406, 0.91728589, 0.91589618],
       [1.        , 0.96458259, 0.9486325 , 0.94151013, 0.93988043],
       [1.        , 0.91745265, 0.91598201, 0.9140279 , 0.91366614],
       ...,
       [1.        , 0.98496258, 0.95651081, 0.95510002, 0.94520708],
       [1.        , 0.94580878, 0.94547733, 0.94100588, 0.94088154],
       [1.        , 0.93688368, 0.93439739, 0.93408488, 0.934075  ]])

In [29]:
def cosine_distance(arr1, arr2):
    sum_yy = (arr2**2).sum()
    sum_xx = (arr1**2).sum()
    sum_xy = arr1.dot(arr2.T)
    return 1 - (sum_xy/da.sqrt(sum_xx*sum_yy))

In [30]:
%%time
cosine_distance(arr_embeddings[0], arr_embeddings[1]).compute()

Wall time: 49 ms


0.24288505

In [31]:
%%time
distance.cosine(arr_embeddings[0], arr_embeddings[1])

Wall time: 3 ms


0.24288511276245117

10\. Работая с исходным файлом в формате `h5`, реализуйте алгоритм подсчета среднего вектора датасета в блочной форме.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных и проведение вычислений над этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент. При работе с `h5` вы можете работать с массивами `numpy.array`. Для итерации по сегментам файла допускается использование циклов.

Сравните время и результаты решения работы вашего алгоритма с реализацией поиска среднего вектора из `dask`. 

In [32]:
dset.shape

(1200000, 312)

In [33]:
%%time

with h5py.File(
    "recipe_embeddings.h5",
    "r"
) as hdf:
    M, N = arr_embeddings.shape
    step = int(M / 12)
    all_sum = np.zeros(N) # sum для каждого куска
    all_count = np.zeros(N) # count для каждого куска
    for i in range(0, 12):
        dset = hdf['embeddings'][i*step:step*(i+1), 0:N]
        all_sum += dset.sum(axis=0)
        all_count += dset.shape[0]
        
mean = all_sum / all_count   

Wall time: 504 ms


In [34]:
%%time

arr_embeddings.mean(axis=0).compute()

Wall time: 433 ms


array([ 9.75612784e-04, -3.39259394e-02,  4.66551706e-02, -8.74475539e-02,
       -9.71175730e-03,  5.51533466e-03,  2.71453876e-02,  3.75371426e-02,
       -2.35138573e-02, -1.10451467e-02,  2.61558387e-02,  8.86423141e-03,
        1.82164591e-02,  5.94256781e-02,  2.72058602e-02, -6.64571347e-03,
        3.04658785e-02,  1.03974873e-02,  1.93780214e-02,  1.43449605e-01,
       -2.46166927e-03, -5.94153954e-03, -3.18063162e-02, -4.10589352e-02,
        8.61881673e-02,  2.80387234e-02, -2.43213642e-02, -4.59721265e-03,
        1.09601039e-02,  3.78381982e-02, -1.06729176e-02, -1.57454032e-02,
        1.08158973e-03, -2.18519717e-02,  7.02829752e-03,  5.17747998e-02,
       -9.01942549e-04, -3.80986892e-02, -7.50485957e-02,  1.78839490e-02,
       -6.01359382e-02,  1.52847767e-01,  7.50525221e-02, -3.78862694e-02,
       -2.17200220e-02,  3.46802338e-03,  3.78257819e-02, -4.40568440e-02,
        4.40716706e-02, -3.56613547e-02,  8.02603643e-03, -3.51646021e-02,
       -4.23886590e-02,  

In [35]:
%%time

with h5py.File("recipe_embeddings.h5", "r"
) as hdf3:
    r, N = (12000, 312)
    summ = np.zeros(N)
    count = np.zeros(N)
    for m in range(r, M + r, r):
        dset = hdf3["embeddings"][m-r:m, 0:N]
        summ += dset.sum(axis=0)
        count += dset.shape[0]

mean = (summ / count)

Wall time: 555 ms
